# Penerapan Konsep Ensemble Stacking (Tugas 10)

##Prepocessing Data

In [6]:
#import Library
import pandas as pd
import numpy as np

In [7]:
# mengakses data 
dataset_url = "https://raw.githubusercontent.com/triasmidwi/dataset/main/credit_score.csv"
# membaca data
df = pd.read_csv(dataset_url)
#menampilkan 5 data pertama
df.head()

,Unnamed: 0,kode_kontrak,pendapatan_setahun_juta,kpr_aktif,durasi_pinjaman_bulan,jumlah_tanggungan,rata_rata_overdue,risk_rating
0,1,AGR-000001,295,YA,48,5,61 - 90 days,4
1,2,AGR-000011,271,YA,36,5,61 - 90 days,4
2,3,AGR-000030,159,TIDAK,12,0,0 - 30 days,1
3,4,AGR-000043,210,YA,12,3,46 - 60 days,3
4,5,AGR-000049,165,TIDAK,36,0,31 - 45 days,2


In [8]:
#memisahkan kolom label/kelas
df_risk_rating = pd.DataFrame(df, columns = ['risk_rating'])
df_risk_rating.head()

,risk_rating
0,4
1,4
2,1
3,3
4,2


In [9]:
#mengambil kolom yang sudah agak fiks
df_without_column_for_convert = pd.DataFrame(df, columns = ['kode_kontrak','pendapatan_setahun_juta','durasi_pinjaman_bulan','jumlah_tanggungan'])
df_without_column_for_convert.head()

,kode_kontrak,pendapatan_setahun_juta,durasi_pinjaman_bulan,jumlah_tanggungan
0,AGR-000001,295,48,5
1,AGR-000011,271,36,5
2,AGR-000030,159,12,0
3,AGR-000043,210,12,3
4,AGR-000049,165,36,0


In [10]:
#memproses kolom 'rata-rata_overdue'
df_ratarata=pd.get_dummies(df['rata_rata_overdue'])
df_ratarata.head()

,0 - 30 days,31 - 45 days,46 - 60 days,61 - 90 days,> 90 days
0,0,0,0,1,0
1,0,0,0,1,0
2,1,0,0,0,0
3,0,0,1,0,0
4,0,1,0,0,0


In [11]:
#memproses kolom kpr_aktif menjadi data biner
df_kpr_aktif=pd.get_dummies(df['kpr_aktif'])
df_kpr_aktif

,TIDAK,YA
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0
...,...,...
895,0,1
896,0,1
897,1,0
898,1,0


In [12]:
#menggabungkan kolom2 yang sudah diproses
df_new = pd.concat([df_without_column_for_convert, df_kpr_aktif,df_ratarata], axis=1)
df_new.head()

,kode_kontrak,pendapatan_setahun_juta,durasi_pinjaman_bulan,jumlah_tanggungan,TIDAK,YA,0 - 30 days,31 - 45 days,46 - 60 days,61 - 90 days,> 90 days
0,AGR-000001,295,48,5,0,1,0,0,0,1,0
1,AGR-000011,271,36,5,0,1,0,0,0,1,0
2,AGR-000030,159,12,0,1,0,1,0,0,0,0
3,AGR-000043,210,12,3,0,1,0,0,1,0,0
4,AGR-000049,165,36,0,1,0,0,1,0,0,0


In [13]:
#melakukan proses minmax scaler pada 3 kolom
from sklearn.preprocessing import MinMaxScaler
df_for_minmax_scaler=pd.DataFrame(df_new, columns = ['pendapatan_setahun_juta','durasi_pinjaman_bulan','jumlah_tanggungan'])
df_for_minmax_scaler.to_numpy()
scaler = MinMaxScaler()
df_hasil_minmax_scaler=scaler.fit_transform(df_for_minmax_scaler)

In [14]:
df_hasil_minmax_scaler = pd.DataFrame(df_hasil_minmax_scaler,columns = ['pendapatan_setahun_juta','durasi_pinjaman_bulan','jumlah_tanggungan'])
df_hasil_minmax_scaler

,pendapatan_setahun_juta,durasi_pinjaman_bulan,jumlah_tanggungan
0,0.978261,1.000000,0.833333
1,0.873913,0.666667,0.833333
2,0.386957,0.000000,0.000000
3,0.608696,0.000000,0.500000
4,0.413043,0.666667,0.000000
...,...,...,...
895,0.182609,1.000000,0.833333
896,0.217391,1.000000,0.333333
897,0.417391,0.333333,0.333333
898,0.547826,1.000000,0.000000


In [15]:
#menghilangkan 3 kolom sebelum di minmax
df_drop_column_for_minmaxscaler=df_new.drop(['pendapatan_setahun_juta','durasi_pinjaman_bulan','jumlah_tanggungan'], axis=1)
df_drop_column_for_minmaxscaler

,kode_kontrak,TIDAK,YA,0 - 30 days,31 - 45 days,46 - 60 days,61 - 90 days,> 90 days
0,AGR-000001,0,1,0,0,0,1,0
1,AGR-000011,0,1,0,0,0,1,0
2,AGR-000030,1,0,1,0,0,0,0
3,AGR-000043,0,1,0,0,1,0,0
4,AGR-000049,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
895,AGR-010739,0,1,0,0,0,0,1
896,AGR-010744,0,1,0,0,1,0,0
897,AGR-010758,1,0,1,0,0,0,0
898,AGR-010775,1,0,0,1,0,0,0


In [16]:
#menggabungkan data yang sudah fix
df_new2 = pd.concat([df_drop_column_for_minmaxscaler,df_hasil_minmax_scaler,df_risk_rating], axis=1)
df_new2

,kode_kontrak,TIDAK,YA,0 - 30 days,31 - 45 days,46 - 60 days,61 - 90 days,> 90 days,pendapatan_setahun_juta,durasi_pinjaman_bulan,jumlah_tanggungan,risk_rating
0,AGR-000001,0,1,0,0,0,1,0,0.978261,1.000000,0.833333,4
1,AGR-000011,0,1,0,0,0,1,0,0.873913,0.666667,0.833333,4
2,AGR-000030,1,0,1,0,0,0,0,0.386957,0.000000,0.000000,1
3,AGR-000043,0,1,0,0,1,0,0,0.608696,0.000000,0.500000,3
4,AGR-000049,1,0,0,1,0,0,0,0.413043,0.666667,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...
895,AGR-010739,0,1,0,0,0,0,1,0.182609,1.000000,0.833333,5
896,AGR-010744,0,1,0,0,1,0,0,0.217391,1.000000,0.333333,3
897,AGR-010758,1,0,1,0,0,0,0,0.417391,0.333333,0.333333,1
898,AGR-010775,1,0,0,1,0,0,0,0.547826,1.000000,0.000000,2


In [17]:
df_new2.shape

(900, 12)

## Traning model

In [18]:
#mengambil beberapa kolom
X=df_new2.iloc[:,1:11].values
y=df_new2.iloc[:,11].values

In [19]:
#melakukan split training dataset
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

## proses data dengan model

In [25]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
#
# Create a Randomforest classifier
#
forest = RandomForestClassifier(n_estimators=100,
                                random_state=123)
#
# Create a XGBoost classifier
#
boost = XGBClassifier(random_state=123, verbosity=0, use_label_encoder=False)
#
# Create a Logistic regression classifier
#
nbclassifier = GaussianNB()
#
# Create a stacking classifier
#
estimators = [
     ('rf', forest),
     ('xgb', boost)
]
sclf = StackingClassifier(estimators=estimators,
                            final_estimator=nbclassifier,
                            cv=10)
#
# Fit the random forest classifier; Print the scores
#
forest.fit(X_train, y_train)
print(f"Random forest model training Accuracy: {forest.score(X_train, y_train):0.2f}")
print(f"Random forest model test Accuracy: {forest.score(X_test, y_test):0.2f}")
#
# Fit the XGBoost classifier; Print the scores
#
boost.fit(X_train, y_train)
print(f"\nXGBoost model training Accuracy: {boost.score(X_train, y_train):0.2f}")
print(f"XGBoost model test Accuracy: {boost.score(X_test, y_test):0.2f}")
#
# Fit the Stacking classifier; Print the scores
#
sclf.fit(X_train, y_train)
print(f"\nStacking classifier training Accuracy: {sclf.score(X_train, y_train):0.2f}")
print(f"Stacking classifier test Accuracy: {sclf.score(X_test, y_test):0.2f}")

Random forest model training Accuracy: 1.00
Random forest model test Accuracy: 1.00

XGBoost model training Accuracy: 1.00
XGBoost model test Accuracy: 1.00

Stacking classifier training Accuracy: 1.00
Stacking classifier test Accuracy: 1.00
